# Clustering and Scoring Job Relocation Opportunities - Experimental Notebook

Austin Rainwater

---

# Initialization

In [1]:
!pip install --quiet --upgrade sqlalchemy pymysql

import pandas as pd
import numpy as np

from sqlalchemy import (
    create_engine,
    Table,
    Column,
    MetaData,
    String,
    Integer
)

---

# Extract, Transform, Load

Since the data is coming from different sources where the data is formatted differently, and I will need to make several API calls, I'm going to write some standard functions to extract, transform, and load data for various citites.